In [1]:
# imports 

# data handling
import pandas as pd 
import numpy as np 
from datetime import datetime
import scipy 
import math 

# visualizations
import matplotlib.pyplot as plt 
import seaborn as sns